In [ ]:
# Source: https://huggingface.co/learn/nlp-course/en/chapter7/5?fw=pt

In [1]:
!pip install transformers==4.30
!pip install accelerate -U
!pip install rouge_score
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 31.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.30.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Attem

In [2]:
import pandas as pd
import torch
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
import numpy as np
import nltk
import evaluate
rouge_score = evaluate.load("rouge")
nltk.download("punkt")
from transformers import DataCollatorForSeq2Seq
from nltk.tokenize import sent_tokenize

2024-04-25 03:18:48.705671: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 03:18:48.705799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 03:18:48.957295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
train_data = Dataset.from_pandas(pd.read_csv("/kaggle/input/dataset12/hindi_train_final_TFIDF.csv")[["Article", "Summary"]].dropna())
val_data = Dataset.from_pandas(pd.read_csv("/kaggle/input/dataset12/hindi_val_final_TFIDF.csv")[["Article", "Summary"]].dropna())
test_data = Dataset.from_pandas(pd.read_csv("/kaggle/input/dataset12/hindi_test_final_TFIDF.csv")[["Article", "Summary"]].dropna())

In [6]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")

tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [9]:
def tokenize_helper(dat):
    model_inputs = tokenizer(
        dat["Article"],
        truncation = True,
        max_length=1024
    )
    true_sent = tokenizer(dat["Summary"], truncation = True, max_length=1024)
    model_inputs["labels"] = true_sent["input_ids"]
    return model_inputs

tokenized_train = train_data.map(tokenize_helper, batched=True)
tokenized_val = val_data.map(tokenize_helper, batched=True)

Map:   0%|          | 0/18507 [00:00<?, ? examples/s]

Map:   0%|          | 0/1035 [00:00<?, ? examples/s]

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)
tokenized_train = tokenized_train.remove_columns(train_data.column_names)
tokenized_val = tokenized_val.remove_columns(val_data.column_names)

In [11]:
batch_size = 4
epochs = 10
args = Seq2SeqTrainingArguments(
    output_dir = "/kaggle/working/",
    evaluation_strategy = "epoch",
    learning_rate = 1e-3,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    save_total_limit=3,
    predict_with_generate=True,
    num_train_epochs = epochs,
    report_to="none"
)

In [12]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.796600,1.637821,13.200400,4.330500,12.778500,12.866400
2,1.527000,1.567023,14.059900,4.660500,13.519200,13.617000
3,1.320300,1.565878,14.975200,5.673900,14.362600,14.561800
4,1.133200,1.573497,15.524600,5.947200,15.063100,15.214600
5,0.967400,1.639580,14.685200,5.515700,14.174000,14.245200
6,0.809200,1.696684,15.827000,6.217500,15.176900,15.279100
7,0.707500,1.778143,15.258700,5.997600,14.682600,14.748900
8,0.582500,1.857941,16.161800,6.379500,15.541500,15.594100
9,0.508700,1.959070,16.390600,6.632700,15.793900,15.891800
10,0.434400,2.042619,16.222600,6.390600,15.688500,15.721800


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=23140, training_loss=0.9853582190972726, metrics={'train_runtime': 16384.6236, 'train_samples_per_second': 11.295, 'train_steps_per_second': 1.412, 'total_flos': 8.508692666366362e+16, 'train_loss': 0.9853582190972726, 'epoch': 10.0})

In [15]:
trainer.evaluate()

{'eval_loss': 2.042618751525879,
 'eval_rouge1': 16.2226,
 'eval_rouge2': 6.3906,
 'eval_rougeL': 15.6885,
 'eval_rougeLsum': 15.7218,
 'eval_runtime': 76.4527,
 'eval_samples_per_second': 13.538,
 'eval_steps_per_second': 1.7,
 'epoch': 10.0}

In [17]:
temp = tokenizer(test_data["Article"][0], truncation = True, padding=True, return_tensors='pt').input_ids.to(device)
out = model.generate(temp, max_length=150, num_beams=4, early_stopping=True) 
print(tokenizer.decode(out[0], skip_special_tokens=True))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


उल्कापिंड जब धरती पर गिरते हैं, तो इनकी चमक इतनी ज्यादा होती है कि लगभग 200 से 300 किलोमीटर के दायरे के लोग आसमान में इसे देख सकते हैं। हालांकि इस घटना ने सबको आश्चर्य में डाल दिया है। दरअसल, उल्कापिंड जब धरती पर गिरते हैं, तो इनकी चमक इतनी ज्यादा होती है कि लगभग 200 से 300 किलोमीटर के दायरे के लोग आसमान में इसे देख सकते हैं।
